In [1]:
import napari
%gui qt5

In [2]:
import brainlit
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.viz.swc import *
import numpy as np
from skimage import io

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/python_jsonschema_objects/__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


## Loading entire neuron from AWS 


`napari.components.viewer_model.ViewerModel.add_swc` does this via the following functions in the `napari.layers.swc.swc` module
1. `swc.read_s3` to read the s3 file into a pd.DataFrame
2. `swc.read_swc` to read the swc file into a pd.DataFrame
3. `swc.generate_df_subset` creates a smaller subset of the original dataframe with coordinates in img space
4. `swc.swc_to_voxel` to convert the coordinates from spatial to voxel coordinates
5. `swc.df_to_graph` to convert the DataFrame into a netwrokx.DiGraph
6. `swc.graph_to_paths` to convert from a graph into a list of paths
7. `ViewerModel.add_shapes` to add the paths as a shape layer into the napari viewer

### 1. `read_s3`
This function parses the swc file into a pd.DataFrame. Each row is a vertex in the swc file with the following information: 

`sample number`

`structure identifier`

`x coordinate`

`y coordinate`

`z coordinate`

`radius of dendrite`

`sample number of parent`

The coordinates are given in spatial units of micrometers ([swc specification](http://www.neuronland.org/NLMorphologyConverter/MorphologyFormats/SWC/Spec.html))

In [3]:
s3_path = "s3://mouse-light-viz/precomputed_volumes/brain1_segments"
seg_id = 2
mip = 1
df = read_s3(s3_path, seg_id, mip)
df.head()

Downloading: 100%|██████████| 1/1 [00:00<00:00, 22.36it/s]


,sample,structure,x,y,z,r,parent
0,1,0,4713.0,4470.0,3857.0,1.0,-1
1,4,192,4721.0,4445.0,3849.0,1.0,1
2,7,64,4723.0,4446.0,3851.0,1.0,4
3,8,0,4728.0,4449.0,3852.0,1.0,7
4,14,0,4746.0,4445.0,3858.0,1.0,8


### 4. `swc_to_voxel`

If we want to overlay the swc file with a corresponding image, we need to make sure that they are in the same coordinate space. Because an image in an array of voxels, it makes sense to convert the vertices in the dataframe from spatial units into voxel units.

Given the `spacing` (spatial units/voxel) and `origin` (spatial units) of the image, `swc_to_voxel` does the conversion by using the following equation:

$voxel = \frac{spatial - origin}{spacing}$

In [4]:
spacing = np.array([0.29875923,0.3044159,0.98840415])
origin = np.array([70093.276,15071.596,29306.737])

df_voxel = swc_to_voxel(df=df, spacing=spacing, origin=origin)
df_voxel.head()

,sample,structure,x,y,z,r,parent
0,1,0,-218839,-34826,-25748,1.0,-1
1,4,192,-218813,-34908,-25756,1.0,1
2,7,64,-218806,-34905,-25754,1.0,4
3,8,0,-218789,-34895,-25753,1.0,7
4,14,0,-218729,-34908,-25747,1.0,8


### 5. `df_to_graph`
A neuron is a graph with no cycles (tree). While napari does not support displaying graph objects, it can display multiple paths. 

The DataFrame already contains all the possible edges in the neurons. Each row in the DataFrame is an edge. For example, from the above we can see that `sample 2` has `parent 1`, which represents edge `(1,2)`. `sample 1` having `parent -1` means that `sample 1` is the root of the tree.

`swc.df_to_graph` reads DataFrame and converts it into a networkx directional graph.

In [5]:
G = df_to_graph(df)
print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))
print('\n')
print('Sample 1 coordinates (x,y,z)')
print(G.nodes[1]['x'],G.nodes[1]['y'],G.nodes[1]['z'])

Number of nodes: 1650
Number of edges: 1649


Sample 1 coordinates (x,y,z)
4713 4470 3857


In [7]:
dir(G)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adj',
 '_node',
 '_pred',
 '_succ',
 'add_edge',
 'add_edges_from',
 'add_node',
 'add_nodes_from',
 'add_weighted_edges_from',
 'adj',
 'adjacency',
 'adjlist_inner_dict_factory',
 'adjlist_outer_dict_factory',
 'clear',
 'copy',
 'degree',
 'edge_attr_dict_factory',
 'edge_subgraph',
 'edges',
 'get_edge_data',
 'graph',
 'graph_attr_dict_factory',
 'has_edge',
 'has_node',
 'has_predecessor',
 'has_successor',
 'in_degree',
 'in_edges',
 'is_directed',
 'is_multigraph',
 'name',
 'nbunch_iter',
 'neighbors',
 'node_attr_dict_factory',
 'node_dict_factory',
 'n

In [ ]:
class Points():
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y
    
    def get_z(self):
        return self.z
    

def get_voxel_info(df_voxel, sample_num):
    for _, row in df_voxel.iterrows():
        if int(row['sample']) == sample_num:
            return Points(row['x'],  row['y'], row['z'])
    
    return None  # never reached


def get_k_successors(seg_id, vertex_id, depth):
    s3_path = "s3://mouse-light-viz/precomputed_volumes/brain1_segments"
    mip = 1
    
    spacing = np.array([0.29875923,0.3044159,0.98840415])
    origin = np.array([70093.276,15071.596,29306.737])
    
    df = read_s3(s3_path, seg_id, mip)
    df_voxel = swc_to_voxel(df=df, spacing=spacing, origin=origin)

    G = df_to_graph(df_voxel)
    
    from collections import deque
    
    unvisited_nodes = deque([vertex_id])
    visited_nodes = []
    visited_points = []
    
    while unvisited_nodes and depth > 0:
        depth -= 1
        
        current_node = unvisited_nodes.popleft()
        
        if current_node in visited_nodes:
            continue
            
        visited_nodes.append(current_node)
        visited_points.append(get_voxel_info(df_voxel, current_node))
        
        
        for child_node in G.successors(current_node):
            unvisited_nodes.append(child_node)
            
    return visited_nodes, visited_points
            

nodes, points = get_k_successors(2,7,5)

In [ ]:
print(nodes)
print(visted)

### 6. `graph_to_paths`
This function takes in a graph and returns a list of non-overlapping paths. The union of the paths forms the graph.

The algorithm works by:

1. Find longest path in the graph ([networkx.algorithms.dag.dag_longest_path](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.dag.dag_longest_path.html))
2. Remove longest path from graph
3. Repeat steps 1 and 2 until there are no more edges left in the graph

In [ ]:
paths = graph_to_paths(G=G)
print(f"The graph was decomposed into {len(paths)} paths")

### 6. `ViewerModel.add_shapes`
napari displays "layers". The most common layer is the image layer. In order to display the neuron, we use `path` from the [shapes](https://napari.org/tutorials/shapes) layer

In [ ]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', name='Skeleton 2')